In [ ]:
import time
import datetime
import hf_hydrodata as hf
import xarray as xr

def get_snowtel_data(site_id, start_date, num_days):
    """
    Get snowtel average temp data from one site for num_days starting at start_date.
    Returns: (data, x, y) where x,y are conus2 grid location of the site.
    """
    site_filter_options = {"dataset": "snotel", "variable": "air_temp", "site_id": site_id}
    site_path = hf.get_paths(site_filter_options) [0]
    ds = xr.open_dataset(site_path)
    lat = ds.attrs["latitude"]
    lon = ds.attrs["longitude"]
    (x, y) = hf.to_ij("conus2", lat, lon)
    (x, y) = hf.to_meters("conus2", lat, lon)
    x = int(x/1000)
    y = int(y/1000)
    print(x, y, lat, lon)
    da = ds["temp_avg"]
    end_date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days=num_days-1)).strftime("%Y-%m-%d")
    data = da.sel(date=slice(start_date, end_date)).values
    data = data + 273.5
    return (data, x, y)

def print_site_cw3e_differences(site_id, start_date, num_days, tolerance):
    """
    Print the difference in average temperature for the observation site compared with CW3E data from num_days from start_date
    Only print values that diff by more than 1 degree Kelvin.
    """
    (site_data, x, y) = get_snowtel_data(site_id, start_date, num_days)
    print(site_data.shape)
    grid_bounds = [x, y, x + 1, y + 1]
    end_time = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days=num_days)
    cw3e_filter_options = {
        "dataset": "CW3E", "variable": "air_temp", "temporal_resolution": "daily", "aggregation":"mean", 
        "grid_bounds": grid_bounds, "start_time": start_date, "end_time":end_time}
    cw3e_data = hf.get_gridded_data(cw3e_filter_options)
    num_diffs = 0
    for i in range(0, num_days):
        diff = site_data[i] - cw3e_data[i, 0, 0]
        if abs(diff) > tolerance:
            num_diffs = num_diffs + 1
            date = (datetime.datetime.strptime(start_date, "%Y-%m-%d") + datetime.timedelta(days = i)).strftime("%Y-%m-%d")
            print(f"{site_id} {date} Site {site_data[i]}, CW3E {cw3e_data[i, 0, 0]} DIFF {diff}")
    print(f"{num_diffs} days outside the {tolerance} degrees")

site_ids = ["350:WY:SNTL", "347:MT:SNTL", "368:UT:SNTL", "396:UT:SNTL", "913:CO:SNTL", "2090:AR:SCAN"]
site_ids = ["350:WY:SNTL"]
site_ids = ["1000:OR:SNTL"]

start_time = "2006-01-01"
num_days = 365
for site_id in site_ids:
    print_site_cw3e_differences(site_id, start_time, num_days, 5)


